In [2]:
import os
import h5py
import torch_geometric as pyg

In [11]:
h5py.__version__

'3.5.0'

In [17]:
import yaml
yaml.__version__

'6.0'

In [21]:
with h5py.File('/home/jupyter/the_hard_way/incorporated/forkane/data/raw/DNS1_00116000.h5') as file:
    print(file.keys())
    x = file['/grad_filt_8'][:]
    y = file['/filt_8'][:]

<KeysViewHDF5 ['filt_8', 'filt_grad_8', 'grad_filt_8']>


In [22]:
grid_shape = x.shape
grid_shape

(65, 33, 33)

In [14]:
root_path = '/home/jupyter/the_hard_way/incorporated/forkane/data/'
os.listdir(os.path.join(root_path))

['DNS1_00125000.h5',
 'DNS1_00121000.h5',
 'DNS1_00119400.h5',
 'DNS1_00120400.h5',
 'DNS3_00128200.h5',
 'DNS2_00119200.h5',
 'DNS2_00121800.h5',
 'DNS1_00121800.h5',
 'DNS1_00126800.h5',
 'DNS3_00127000.h5',
 'DNS2_00122200.h5',
 'DNS1_00119200.h5',
 'DNS1_00120000.h5',
 'DNS3_00129800.h5',
 'DNS1_00117200.h5',
 'DNS1_00116400.h5',
 'DNS1_00124800.h5',
 'DNS3_00128400.h5',
 'DNS1_00118000.h5',
 'DNS2_00117200.h5',
 'DNS2_00122000.h5',
 'DNS2_00116800.h5',
 'DNS1_00124000.h5',
 'DNS2_00125000.h5',
 'DNS2_00118400.h5',
 'DNS1_00122400.h5',
 'DNS1_00125400.h5',
 'DNS1_00121200.h5',
 'DNS1_00121600.h5',
 'DNS2_00117000.h5',
 'DNS1_00117000.h5',
 'DNS2_00118000.h5',
 'DNS2_00125800.h5',
 'DNS1_00123200.h5',
 'DNS2_00116600.h5',
 'DNS2_00117400.h5',
 'DNS1_00127000.h5',
 'DNS2_00121400.h5',
 'DNS1_00116800.h5',
 'DNS1_00123400.h5',
 'DNS1_00119800.h5',
 'DNS2_00120200.h5',
 'DNS2_00124800.h5',
 'DNS2_00118800.h5',
 'DNS2_00124000.h5',
 'DNS2_00125200.h5',
 'DNS1_00116600.h5',
 'DNS3_001292

In [23]:
with open(os.path.join(root_path, "filenames.yaml"), "r") as f:
    filenames = yaml.safe_load(f)

print(filenames)

['DNS1_00125000.h5', 'DNS1_00121000.h5', 'DNS1_00119400.h5', 'DNS1_00120400.h5', 'DNS3_00128200.h5', 'DNS2_00119200.h5', 'DNS2_00121800.h5', 'DNS1_00121800.h5', 'DNS1_00126800.h5', 'DNS3_00127000.h5', 'DNS2_00122200.h5', 'DNS1_00119200.h5', 'DNS1_00120000.h5', 'DNS3_00129800.h5', 'DNS1_00117200.h5', 'DNS1_00116400.h5', 'DNS1_00124800.h5', 'DNS3_00128400.h5', 'DNS1_00118000.h5', 'DNS2_00117200.h5', 'DNS2_00122000.h5', 'DNS2_00116800.h5', 'DNS1_00124000.h5', 'DNS2_00125000.h5', 'DNS2_00118400.h5', 'DNS1_00122400.h5', 'DNS1_00125400.h5', 'DNS1_00121200.h5', 'DNS1_00121600.h5', 'DNS2_00117000.h5', 'DNS1_00117000.h5', 'DNS2_00118000.h5', 'DNS2_00125800.h5', 'DNS1_00123200.h5', 'DNS2_00116600.h5', 'DNS2_00117400.h5', 'DNS1_00127000.h5', 'DNS2_00121400.h5', 'DNS1_00116800.h5', 'DNS1_00123400.h5', 'DNS1_00119800.h5', 'DNS2_00120200.h5', 'DNS2_00124800.h5', 'DNS2_00118800.h5', 'DNS2_00124000.h5', 'DNS2_00125200.h5', 'DNS1_00116600.h5', 'DNS3_00129200.h5', 'DNS1_00122800.h5', 'DNS2_00116200.h5',

In [ ]:
class Dataset(pyg.data.Dataset):
    
    def __init__(self, root, transform=None, pre_transform=None):
        self._grid_shape = None
        super().__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        with open(os.path.join(self.root, "filenames.yaml"), "r") as stream:
            filenames = yaml.safe_load(stream)
        return filenames

    @property
    def processed_file_names(self):
        return [f"data-{idx}.pt" for idx in range(self.len())]
    
    @property
    def grid_shape(self):
        if not self._grid_shape:
            with h5py.File(self.raw_paths[0], 'r') as file:
                field_val = file['filt_8'][:]
            x_size, y_size, z_size = field_val.shape[0], field_val.shape[1], field_val.shape[2]
            self._grid_shape = (z_size, y_size, x_size)
        return self._grid_shape

    def download(self):
        raise RuntimeError(
            'Data not found. Please download the data at {} and move all files in file.tgz/DATA in {}'.format(
                'https://uftp.fz-juelich.de:9112/UFTP_Auth/rest/access/JUDAC/dc4eef36-1929-41f6-9eb9-c11417be1dcf',
                self.raw_dir))

    def process(self):
        graph = pyg.utils.convert.from_networkx(
            nx.grid_graph(dim=self.grid_shapae)
        )
        undirected_index = graph.edge_index
        
        i = 0
        for raw_path in self.raw_paths:
            
            with h5py.File(raw_path, 'r') as file:
                c = file["/filt_8"][:]
                sigma = file["/filt_grad_8"][:]
                
            data = pyg.data.Data(
                x=torch.tensor(c.reshape(-1,1), dtype=torch.float), 
                edge_index=torch.tensor(undirected_index, dtype=torch.long),
                y=torch.tensor(sigma.reshape(-1,1), dtype=torch.float)
            )

            if self.pre_filter is not None and not self.pre_filter(data):
                continue

            if self.pre_transform is not None:
                data = self.pre_transform(data)

            torch.save(data, os.path.join(self.processed_dir, 'data-{}.pt'.format(i)))
            i += 1

    def get(self, idx):
        data = torch.load(os.path.join(self.processed_dir, 'data-{}.pt'.format(idx)))
        return data
    
    def len(self):
        return len(self.raw_file_names)

In [2]:
"hello world!".split()

['hello', 'world!']

In [5]:
hey = list((0,))
hey.extend([2, 2, 3])
hey

[0, 2, 2, 3]

In [9]:
import numpy as np

np.array(np.arange(16), shape=(4, 4))

TypeError: 'shape' is an invalid keyword argument for array()

In [73]:
x_dim = 65
y_dim = 33
z_dim = 33

In [75]:
array = np.reshape(np.arange(x_dim * y_dim * z_dim), (x_dim, y_dim, z_dim))

In [51]:
arr = np.array([[[0, 1, 2, 3],
                [4, 5, 6, 7],
                [8, 9, 10, 11],
                [12, 13, 14, 15]],
                [[40, 41, 42, 43],
                [44, 45, 46, 47],
                [48, 49, 410, 411],
                [412, 413, 414, 415]]])

In [52]:
arr

array([[[  0,   1,   2,   3],
        [  4,   5,   6,   7],
        [  8,   9,  10,  11],
        [ 12,  13,  14,  15]],

       [[ 40,  41,  42,  43],
        [ 44,  45,  46,  47],
        [ 48,  49, 410, 411],
        [412, 413, 414, 415]]])

In [18]:
arr_right = arr + 1
print(arr_right)
arr_right = arr_right[..., :-1]

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [55]:
left = arr[..., :-1]
left

array([[[  0,   1,   2],
        [  4,   5,   6],
        [  8,   9,  10],
        [ 12,  13,  14]],

       [[ 40,  41,  42],
        [ 44,  45,  46],
        [ 48,  49, 410],
        [412, 413, 414]]])

In [76]:
import numpy as np

def grid_3d_connectivity_matrix(shape):
    
    x_dim, y_dim, z_dim = shape
    
    array = np.reshape(np.arange(x_dim * y_dim * z_dim), (x_dim, y_dim, z_dim))
    
    columnar = (-1, 1)
    
    left = np.reshape(array[..., :-1], columnar)
    right = np.reshape(array[..., 1:], columnar)
    up = np.reshape(array[:, :-1, :], columnar)
    down = np.reshape(array[:, 1:, :], columnar)
    sup = np.reshape(array[1:, ...], columnar)
    inf = np.reshape(array[:-1, ...], columnar)
    
    connectivity = np.concatenate(
        (np.hstack((left, right)),
         np.hstack((right, left)),
         np.hstack((up, down)),
         np.hstack((down, up)),
         np.hstack((sup, inf)),
         np.hstack((inf, sup))),)
    
    return connectivity

In [77]:
c = grid_3d_connectivity_matrix((3, 3, 3))

In [78]:
c

array([[ 0,  1],
       [ 1,  2],
       [ 3,  4],
       [ 4,  5],
       [ 6,  7],
       [ 7,  8],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [15, 16],
       [16, 17],
       [18, 19],
       [19, 20],
       [21, 22],
       [22, 23],
       [24, 25],
       [25, 26],
       [ 1,  0],
       [ 2,  1],
       [ 4,  3],
       [ 5,  4],
       [ 7,  6],
       [ 8,  7],
       [10,  9],
       [11, 10],
       [13, 12],
       [14, 13],
       [16, 15],
       [17, 16],
       [19, 18],
       [20, 19],
       [22, 21],
       [23, 22],
       [25, 24],
       [26, 25],
       [ 0,  3],
       [ 1,  4],
       [ 2,  5],
       [ 3,  6],
       [ 4,  7],
       [ 5,  8],
       [ 9, 12],
       [10, 13],
       [11, 14],
       [12, 15],
       [13, 16],
       [14, 17],
       [18, 21],
       [19, 22],
       [20, 23],
       [21, 24],
       [22, 25],
       [23, 26],
       [ 3,  0],
       [ 4,  1],
       [ 5,  2],
       [ 6,  3],
       [ 7,  4

In [56]:
right = arr[..., 1:]
right

array([[[  1,   2,   3],
        [  5,   6,   7],
        [  9,  10,  11],
        [ 13,  14,  15]],

       [[ 41,  42,  43],
        [ 45,  46,  47],
        [ 49, 410, 411],
        [413, 414, 415]]])

In [61]:
up = arr[:, :-1, :]
up

array([[[  0,   1,   2,   3],
        [  4,   5,   6,   7],
        [  8,   9,  10,  11]],

       [[ 40,  41,  42,  43],
        [ 44,  45,  46,  47],
        [ 48,  49, 410, 411]]])

In [62]:
down = arr[:, 1:, :]
down

array([[[  4,   5,   6,   7],
        [  8,   9,  10,  11],
        [ 12,  13,  14,  15]],

       [[ 44,  45,  46,  47],
        [ 48,  49, 410, 411],
        [412, 413, 414, 415]]])

In [67]:
z_up = arr[1:, ...]
z_up

array([[[ 40,  41,  42,  43],
        [ 44,  45,  46,  47],
        [ 48,  49, 410, 411],
        [412, 413, 414, 415]]])

In [69]:
z_down = arr[:-1, ...]
z_down

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]]])

In [22]:
arr_right.reshape((-1, 1))

array([[ 1],
       [ 2],
       [ 3],
       [ 5],
       [ 6],
       [ 7],
       [ 9],
       [10],
       [11],
       [13],
       [14],
       [15]])

In [24]:
arr_right = arr[..., 1:]
arr_right

array([[ 1,  2,  3],
       [ 5,  6,  7],
       [ 9, 10, 11],
       [13, 14, 15]])

In [25]:
arr_down = arr[1:, ...]
arr_down

array([[ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [27]:
arr_up = arr[:-1, ...]
arr_up

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [30]:
arr_left = arr[..., :-1]
arr_left

array([[ 0,  1,  2],
       [ 4,  5,  6],
       [ 8,  9, 10],
       [12, 13, 14]])

In [31]:
columnar = (-1, 1)

In [32]:
arr_left.reshape(columnar)

array([[ 0],
       [ 1],
       [ 2],
       [ 4],
       [ 5],
       [ 6],
       [ 8],
       [ 9],
       [10],
       [12],
       [13],
       [14]])

In [49]:
connectivity = np.concatenate(
    (np.hstack((arr_left.reshape(columnar), arr_right.reshape(columnar))),
     np.hstack((arr_right.reshape(columnar), arr_left.reshape(columnar))),
     np.hstack((arr_up.reshape(columnar), arr_down.reshape(columnar))),
     np.hstack((arr_down.reshape(columnar), arr_up.reshape(columnar)))),)
     

In [50]:
connectivity.shape

(48, 2)

In [43]:
a = np.hstack((arr_left.reshape(columnar), arr_right.reshape(columnar)))
a

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [14, 15]])

In [46]:
np.concatenate((a, a))

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [14, 15],
       [ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 4,  5],
       [ 5,  6],
       [ 6,  7],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [14, 15]])